<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5A_S7_BERT%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88_U20R50t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT データセットの作成 S7</center>

###<center>リツイート・@mention・URLの除去・短い文(20単語未満)の削除</center>

###<center>5万件のツイートを使用</center>


#0. 作業環境の準備

##0.1. google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2021-11-09


##option. データセットの指定

In [ ]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v6S7_all_forBERT_2021-11-08_U10sentences.csv'

In [ ]:
#INPUT_CSV_v5='/content/drive/MyDrive/研究室/研究_M2/src/Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_2021-11-09_U20R10000.csv'

#1. データセットのロード

In [ ]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

データサイズ： (10000, 14)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,夏場に買ったスニーカーをずっと履いているんですが流石に北風が生地をスースー通り抜けてくるので...,2017-12-20 01:05:36+00:00,ISFJ,I,S,F,J
1,プレミアムフライデーか…何の関係もないのにイベントだけあるからクソめんどくさいんだよな🙃💥,2017-04-27 23:36:47+00:00,ISFP,I,S,F,P
2,あーここまでやったら怒られるだろうなーっつーぎりぎりを探すのが割と好きって話ーまーそんなかん...,2017-12-03 14:04:54+00:00,ISFJ,I,S,F,J
3,スプラトゥーン2までゼルダをやり込もうと思ってたのに、ラスボス倒したらサブクエやる気一気にな...,2017-06-19 03:51:29+00:00,INTJ,I,N,T,J
4,えぇーΣ(´・ｪ・｀；) 自分の場合、家なくなったら色んなもの保管しておけなくなるから困るな...,2017-12-12 08:23:05+00:00,ISFJ,I,S,F,J


#2. データの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [ ]:
#df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df_v0=df_v0
df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,あけおめなのよ。ことよろなのよ。,2017-12-30 12:02:23+00:00,INTJ,I,N,T,J
1,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
2,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
3,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
4,すげー！！耐えた！！ #鉄腕DASH,2017-11-19 10:32:29+00:00,INTJ,I,N,T,J


##2.2. データからリツイートの投稿を除去

In [ ]:
#dfXXX=df['text'].str.extract(r'RT\ @(.*)', expand=True) 
df = df_v0[~df_v0['text'].str.contains(r'RT\ @(.*)')]
#dfXXX = df[~df['text'].str.startswith('RT ')]
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,あけおめなのよ。ことよろなのよ。,2017-12-30 12:02:23+00:00,INTJ,I,N,T,J
1,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
2,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
3,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
4,すげー！！耐えた！！ #鉄腕DASH,2017-11-19 10:32:29+00:00,INTJ,I,N,T,J


##2.3. @mention・URLの削除

In [ ]:
import re
import numpy as np

def remove_MENTIONandURL(text):
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  element_mention=None
  element_URL=None
  for idx, element in enumerate(list_text):
    if re.match(r'(^)(@)[^a-zA-Z0-9_]*', element):
      element_mention=element
    if element.startswith('https:'):
      element_URL=element
      
  #print(element_mention, element_URL)
  if element_mention==None:
    pass
  else:
    list_text.remove(element_mention)
  if element_URL==None:
    pass
  else:
    list_text.remove(element_URL)

  Z=' '.join(list_text)
  
  return Z

df['text'] = df['text'].map(remove_MENTIONandURL)
df.dropna(how='any', axis=0, inplace=True)
df['text']

0                                          あけおめなのよ。ことよろなのよ。
1             CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？
2         中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...
3                   Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。
4                                        すげー！！耐えた！！ #鉄腕DASH
                                ...                        
242878                   肩と背中が痛いと言ったらうちの子に肩たたきをしてもらった  ほっこり
242879      もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体
242880                        こんな時間から声なんて出ないよあほむすめぇぇぇぇ ｹﾞﾎｯ
242881    コンタクトどこ行った？ →私「まさか捨てた？」 →捨ててたわ  ていうかマンガみたいな会話を...
242882    お薬スープ200、角煮100、お寿司100、 ……仕込みが終わった。これでせめて１日位もってくれ。
Name: text, Length: 242870, dtype: object

In [ ]:
df_v3 = df
#df_v3=df.sample(n=100)
df_v3[['text', 'tweet_time', 'conversation_id','MBTI_Type']].head(5)

,text,tweet_time,conversation_id,MBTI_Type
0,あけおめなのよ。ことよろなのよ。,2017-12-30 12:02:23+00:00,9.470752e+17,INTJ
1,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,9.467430e+17,INTJ
2,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,9.346768e+17,INTJ
3,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,9.322774e+17,INTJ
4,すげー！！耐えた！！ #鉄腕DASH,2017-11-19 10:32:29+00:00,9.321949e+17,INTJ


In [ ]:
!pip install janome

     |████████████████████████████████| 19.7 MB 6.4 MB/s 


In [ ]:
from janome.tokenizer import Tokenizer
import collections
from tqdm.notebook import tqdm

import re
import numpy as np

t = Tokenizer()

def remove_ShortText(text):
    one_tweet = "".join(text) #"\n".join(text)
    # print("----------------------------------------")
    # print(one_tweet)
    # print('↓')

    Nan = np.nan
    if len(list(t.tokenize(one_tweet)))> 19:
        #print("長い : ", one_tweet)
        #pass
        return one_tweet
    else:
        #print("短い : ", one_tweet)
        return None

# remove_ShortText(text)
# print()
#QQQ= df_v3['text'][1:100].map(remove_ShortText)
df_v3['text'] = df_v3['text'][:].map(remove_ShortText)
df_v3.dropna(how='any', axis=0, inplace=True)
df_v3['text']

1             CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？
2         中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...
3                   Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。
13        こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...
17        ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...
                                ...                        
242877    ずっと思ってたんだけど、  紅蓮でスキル一新されたのは良いけど、旧コンテンツで使ってたスキル...
242878                   肩と背中が痛いと言ったらうちの子に肩たたきをしてもらった  ほっこり
242879      もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体
242881    コンタクトどこ行った？ →私「まさか捨てた？」 →捨ててたわ  ていうかマンガみたいな会話を...
242882    お薬スープ200、角煮100、お寿司100、 ……仕込みが終わった。これでせめて１日位もってくれ。
Name: text, Length: 133350, dtype: object

In [ ]:
df_v3['text'].head(5)

1         CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？
2     中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...
3               Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。
13    こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...
17    ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...
Name: text, dtype: object

#3. データの保存

##3.1. 手法4までで作成した投稿を更新

In [ ]:
df_all=df_v3
print(f'データサイズ： {df_all.shape}')

FileName=pull_Datetime()
df_all.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6S7_all_%s_U20R50000.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v6S5_all_%s_U20R50000.csv' % FileName)

データサイズ： (133350, 13)
./Assets/Assets_Output/Tweet_Datasets/v6S5_all_2021-11-09_U20R50000.csv


##3.2. データセットS3の作成

In [ ]:
import random
 #データの抽出
df_S=df_all.sample(n=10000)

In [ ]:
FileName=pull_Datetime()
df_S.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_%s_U20R50000.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_%s_U20R50000.csv' % FileName)

./Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_2021-11-09_U20R10000.csv


##3.3. 記録用に各指標のユーザー数を表示

In [ ]:
def num_Type_Index(element):
  num_Users_Type=df_S.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()

  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    1552
I    8448
dtype: int64

label_NorS
N    5815
S    4185
dtype: int64

label_ForT
F    6766
T    3234
dtype: int64

label_JorP
J    3538
P    6462
dtype: int64



#Option 最大単語数の計算

In [ ]:
!pip install transformers

!pip install fugashi
!pip install ipadic

In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df_S['text'].values #sentences = df.sentence.values
  labels= df_S['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)